In [1]:
# HIDDEN
import warnings
# Ignore numpy dtype warnings. These warnings are caused by an interaction
# between numpy and Cython and can be safely ignored.
# Reference: https://stackoverflow.com/a/40846742
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 8)
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
# pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
# HIDDEN
def df_interact(df, nrows=7, ncols=7):
    '''
    Outputs sliders that show rows and columns of df
    '''
    def peek(row=0, col=0):
        return df.iloc[row:row + nrows, col:col + ncols]

    row_arg = (0, len(df), nrows) if len(df) > nrows else fixed(0)
    col_arg = ((0, len(df.columns), ncols)
               if len(df.columns) > ncols else fixed(0))
    
    interact(peek, row=row_arg, col=col_arg)
    print('({} rows, {} columns) total'.format(df.shape[0], df.shape[1]))

def display_df(df, rows=pd.options.display.max_rows,
               cols=pd.options.display.max_columns):
    with pd.option_context('display.max_rows', rows,
                           'display.max_columns', cols):
        display(df)

# License Plates

In [3]:
def set_cols(df):
    df.columns = ['plate', 'time', 'location']
    return df

def clean_location(df):
    locations = (df['location']
     .str.strip('()')
     .str.split(', ', expand=True)
     .rename(columns={0: 'lat', 1: 'lon'})
     .assign(lat=lambda df: df['lat'].astype(float))
     .assign(lon=lambda df: df['lon'].astype(float))
    )
    return (df.assign(lat=locations['lat'], lon=locations['lon'])
            .drop('location', axis=1))

def convert_times(df):
    return df.assign(time=pd.to_datetime(df['time'],
                                         format='%m/%d/%Y %I:%M:%S %p'))

def colorize(df):
    def color(t):
        if t.weekday() >= 6:
            return 'green' # Weekend
        if t.hour >= 6 and t.hour <= 17:
            return 'blue' # Weekday daytime
        return 'red' # Weekday evening
    return df.assign(color=df['time'].apply(color))

In [12]:
plates = (pd.read_csv('../data/all-lprs.csv.gz')
          .pipe(set_cols)
          .pipe(clean_location)
          .pipe(convert_times)
          .pipe(colorize))
plates

In [6]:
import folium
def map_for_license(license):
    m = folium.Map(location=[37.798, -122.276], zoom_start=12)
    for row in plates.loc[plates['plate'] == license].itertuples():
        folium.Marker(
            location=[row.lat, row.lon],
            popup=f'{row.time}',
#             icon=folium.Icon(color='blue')
            icon=folium.Icon(color=row.color)
        ).add_to(m)
    return m

### Jean Quan

In [8]:
map_for_license('6FCH845')

### Fire Chief

In [9]:
map_for_license('1328354')

### Random People

In [10]:
map_for_license('5AJG153')

In [11]:
map_for_license('6UZA652')